In [3]:
import pandas as pd
import numpy as np
import duckdb

con = duckdb.connect('/home/rgc/projects/dlt/Homework/taxi_pipeline.duckdb', read_only=True)

In [4]:
con.sql("DESCRIBE")

┌───────────────┬──────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│   database    │        schema        │        name         │                                                                                                                   column_names                                                                                                                   │                                                                                       column_types                                                                                       │ te

## What is the start date and end date of the dataset?

In [5]:
con.sql("SELECT MIN(trip_pickup_date_time), MAX(trip_pickup_date_time) FROM taxi_dataset.taxi_trip")

┌────────────────────────────┬────────────────────────────┐
│ min(trip_pickup_date_time) │ max(trip_pickup_date_time) │
│  timestamp with time zone  │  timestamp with time zone  │
├────────────────────────────┼────────────────────────────┤
│ 2009-06-01 13:43:00+02     │ 2009-06-30 23:05:00+02     │
└────────────────────────────┴────────────────────────────┘

In [6]:
# Python alternative using pandas
import pandas as pd
df = pd.read_sql("SELECT MIN(trip_pickup_date_time) as start_date, MAX(trip_pickup_date_time) as end_date FROM taxi_dataset.taxi_trip", con)
df

/tmp/ipykernel_91065/3721844661.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT MIN(trip_pickup_date_time) as start_date, MAX(trip_pickup_date_time) as end_date FROM taxi_dataset.taxi_trip", con)


,start_date,end_date
0,2009-06-01 11:43:00+00:00,2009-06-30 21:05:00+00:00


## What proportion of trips are paid with credit card?

In [ ]:
# Vérifier les types de paiement disponibles
con.sql("SELECT DISTINCT payment_type FROM taxi_dataset.taxi_trip ORDER BY payment_type")

┌──────────────┐
│ payment_type │
│   varchar    │
├──────────────┤
│ CASH         │
│ Cash         │
│ Credit       │
└──────────────┘

In [8]:
# Python alternative for payment types
import pandas as pd
df = pd.read_sql("SELECT DISTINCT payment_type FROM taxi_dataset.taxi_trip ORDER BY payment_type", con)
df

/tmp/ipykernel_91065/2673560475.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT DISTINCT payment_type FROM taxi_dataset.taxi_trip ORDER BY payment_type", con)


,payment_type
0,CASH
1,Cash
2,Credit


In [9]:
# Calculer la proportion de trajets payés avec une carte de crédit
con.sql("""
WITH trip_stats AS (
    SELECT 
        COUNT(*) as total_trips,
        SUM(CASE WHEN payment_type = 'Credit' THEN 1 ELSE 0 END) as credit_trips
    FROM taxi_dataset.taxi_trip
)
SELECT 
    total_trips,
    credit_trips,
    ROUND(100.0 * credit_trips / total_trips, 2) as proportion_percent
FROM trip_stats
""")

┌─────────────┬──────────────┬────────────────────┐
│ total_trips │ credit_trips │ proportion_percent │
│    int64    │    int128    │       double       │
├─────────────┼──────────────┼────────────────────┤
│        1000 │          257 │               25.7 │
└─────────────┴──────────────┴────────────────────┘

In [9]:
# Python equivalent for proportion calculation
import pandas as pd
query = """
SELECT
    COUNT(*) as total_trips,
    SUM(CASE WHEN payment_type = 'Credit' THEN 1 ELSE 0 END) as credit_trips
FROM taxi_dataset.taxi_trip
"""
stats = pd.read_sql(query, con)
stats['proportion_percent'] = (stats.credit_trips / stats.total_trips * 100).round(2)
stats

/tmp/ipykernel_91065/1913112695.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stats = pd.read_sql(query, con)


,total_trips,credit_trips,proportion_percent
0,1000,257,25.7


25,7% des paiement ont été efectués par carte bancaire

## What is the total amount of money generated in tips?

In [12]:
con.sql("""
SELECT 
    ROUND(SUM(tip_amt),1) as total_tips
FROM 
    taxi_dataset.taxi_trip
""")

┌────────────┐
│ total_tips │
│   double   │
├────────────┤
│      553.9 │
└────────────┘

In [13]:
# Python alternative for total tips
import pandas as pd
df = pd.read_sql("SELECT SUM(tip_amt) as total_tips FROM taxi_dataset.taxi_trip", con)
df.total_tips = df.total_tips.round(1)
df

/tmp/ipykernel_91065/2528386892.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT SUM(tip_amt) as total_tips FROM taxi_dataset.taxi_trip", con)


,total_tips
0,553.9
